In [1]:
pip install torch torchvision torchaudio

  Using cached torch-2.0.1-cp39-cp39-win_amd64.whl (172.4 MB)
Note: you may need to restart the kernel to use updated packages.


In [90]:
pip install tf-slim

In [91]:
%matplotlib inline

import gym
import numpy as np
import matplotlib
import itertools
import sys
import torch
import collections
import tensorflow.compat.v1 as tf
from tf_slim.layers import layers as _layers

if "https://github.com/dennybritz/reinforcement-learning/blob/master/lib/envs/cliff_walking.py" not in sys.path:
    sys.path.append("https://github.com/dennybritz/reinforcement-learning/blob/master/lib/envs/cliff_walking.py")

from Lib.envs.cliff_walking import CliffWalkingEnv
from Lib import plotting

matplotlib.style.use('ggplot')

In [92]:
tf.compat.v1.disable_eager_execution()

In [93]:
env=CliffWalkingEnv()

In [123]:
class PolicyEstimator():
    #Policy Function Approximator
    
    def __init__(self,learning_rate=0.01,scope="policy_estimator"):
        with tf.variable_scope(scope):
            self.state=tf.placeholder(dtype=tf.int32,name="state")
            self.action=tf.placeholder(dtype=tf.int32,name="action")
            self.target=tf.placeholder(dtype=tf.int32,name="target")
            
            state_one_hot=tf.one_hot(self.state,int(env.observation_space.n))
            self.output_layer=_layers.fully_connected(inputs=tf.expand_dims(state_one_hot,0),
                                                                num_outputs=env.action_space.n,
                                                                activation_fn=None,
                                                                weights_initializer=tf.zeros_initializer)
            self.action_probs=tf.squeeze(tf.nn.softmax(self.output_layer))
            self.picked_action_prob=tf.gather(self.action_probs,self.action)
            
            self.loss=-tf.log(self.picked_action_prob)*self.target
            
            self.optimizer=tf.train.AdamOptimizer(learning_rate=learning_rate)
            self.train_op=self.optimizer.minimize(self.loss
                                                 ,global_step=tf.estimator.framework.get_global_step())
            
    def predict(self,state,sess=None):
        sess=sess or tf.get_default_session()
        return sess.run(self.action_probs,{self.state:state})
    
    def update(self,state,target,action,sess=None):
        sess=sess or tf.get_default_session()
        feed_dict={self.state:state,self.target:target,self.action:action}
        _,loss=sess.run([self.train_op,self.loss],feed_dict)
        return loss

In [124]:
class ValueEstimator():
    #Value Function Approximator
    
    def __init__(self,learning_rate=0.1,scope="Value_estimator"):
        with tf.variable_scope(scope):
            self.state=tf.placeholder(tf.int32,"state")
            self.target=tf.placeholder(tf.int32,"target")
            
            state_one_hot=tf.one_hot(self.state,int(env.observation_space.n))
            self.output_layer=_layers.fully_connected(inputs=tf.expand_dims(state_one_hot,0)
                                                       ,num_outputs=1
                                                       ,activation_fn=None
                                                       ,weights_initializer=tf.zeros_initializer)
            
            self.value_estimate=tf.squeeze(self.output_layer)
            self.loss=tf.squared_difference(self.value_estimate,self.target)
            
            self.optimizer=tf.train.AdamOptimizer(learning_rate=learning_rate)
            self.train_op=self.optimizer.minimize(self.loss
                                                  ,global_step=tf.estimator.framework.get_global_step())
            
    def predict(self,state,sess=None):
        sess=sess or tf.get_default_session()
        return sess.run(self.value_estimate,{self.state:state})
    
    def update(self,state,target,sess=None):
        sess=sess or tf.get_default_session()
        feed_dict={self.state:state,self.target:target}
        _,loss=sess.run([self.train_op,self.loss],feed_dict)
        return loss

In [125]:
def reinforce(env,estimator_policy,estimator_value,num_episodes,discount_factor=1.0):
    stats=plotting.EpisodeStats(episode_lengths=np.zeros(num_episodes)
                               ,episode_rewards=np.zeros(num_episodes))
    
    Transition=collections.namedtuple("Transition",["state","action","reward","next_state","done"])
    
    for i_episode in range(num_episodes):
        state=env.reset()
        episode=[]
        
        for t in itertools.count():
            action_probs=estimator_policy.predict(state)
            action=np.random.choice(np.arange(len(action_probs)),p=action_probs)
            next_state,reward,done,_=env.step(action)
            
            episode.append(Transition(state=state,action=action,reward=reward,next_state=next_state,done=done))
            
            stats.episode_rewards[i_episode]+=reward
            stats.episode_lengths=t
            
            print("\rstep {} @ Episode {}/{} ({})".format(t,i_episode+1,num_episodes,stats.episode_rewards[i_episode-1]),end="")
            
            if done:
                break
            
            state=next_state
            
            for t,transition in enumerate(episode):
                total_return=sum(discount_factor**i*t.reward for i,t in enumerate(episode[t:]))
                baseline_value=estimator_value.predict(transition.state)
                advantage=total_return-baseline_value
                estimator_value.update(transition.state,total_return)
                estimator_poilcy.update(transition.state,advantage,transition.action)
                
    return stats

In [126]:
tf.reset_default_graph()

global_step=tf.Variable(0,name="global_step",trainable=False)
policy_estimator=PolicyEstimator()
value_estimator=ValueEstimator()

with tf.session() as sess:
    sess.run(tf.initialize_all_variables())
    stats=reinforce(env,policy_estimator,value_estimator,2000,discount_factor=1.0)

ValueError: The last dimension of the inputs to `Dense` should be defined. Found `None`.

In [ ]:
plotting.plot_episode_stats(stats,smoothing_window=25)